# Rolling Update Tests

Check rolling updates function as expected.

In [66]:
import json
import time

Before we get started we'd like to make sure that we're making all the changes in a new blank namespace of the name `seldon`

In [67]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


## Change Image

We'll want to try modifying an image and seeing how the rolling update performs the changes.

We'll first create the following model:

In [68]:
%%writefile resources/fixed_v1.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 3

Overwriting resources/fixed_v1.yaml


Now we can run that model and wait until it's released

In [69]:
!kubectl apply -f resources/fixed_v1.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed created


Let's confirm that the state of the model is ready

In [70]:
!kubectl wait sdep/fixed --for=condition=ready --timeout=120s -n seldon

seldondeployment.machinelearning.seldon.io/fixed condition met


In [71]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

{"data":{"names":[],"ndarray":[1,2,3,4]},"meta":{"requestPath":{"classifier":"seldonio/fixed-model:0.1"}}}


Now we can modify the model by providing a new image name, using the following config file:

In [72]:
%%writefile resources/fixed_v2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 3

Overwriting resources/fixed_v2.yaml


In [73]:
!kubectl apply -f resources/fixed_v2.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed configured


Now let's actually send a couple of requests to make sure that there are no failed requests as the rolling update is performed

In [74]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 3:
        break
    time.sleep(1)
print("Rollout Success")

Rollout Success


In [75]:
!kubectl delete -f resources/fixed_v1.yaml -n seldon

seldondeployment.machinelearning.seldon.io "fixed" deleted


## Change Replicas (no rolling update)

We'll want to try modifying number of replicas and no rolling update is needed.

We'll first create the following model:

In [76]:
%%writefile resources/fixed_v1_rep2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 2

Overwriting resources/fixed_v1_rep2.yaml


Now we can run that model and wait until it's released

In [77]:
!kubectl apply -f resources/fixed_v1_rep2.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed created


Let's confirm that the state of the model is Available

In [78]:
!kubectl wait sdep/fixed --for=condition=ready --timeout=120s -n seldon

seldondeployment.machinelearning.seldon.io/fixed condition met


In [79]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

{"data":{"names":[],"ndarray":[1,2,3,4]},"meta":{"requestPath":{"classifier":"seldonio/fixed-model:0.1"}}}


Now we can modify the model by providing a new image name, using the following config file:

In [80]:
%%writefile resources/fixed_v1_rep4.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 4

Overwriting resources/fixed_v1_rep4.yaml


In [81]:
!kubectl apply -f resources/fixed_v1_rep4.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed configured


Now let's actually send a couple of requests to make sure that there are no failed requests as the rolling update is performed

In [82]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 4:
        break
    time.sleep(1)
print("Rollout Success")

Rollout Success


Now downsize back to 2

In [83]:
!kubectl apply -f resources/fixed_v1_rep2.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed configured


In [84]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 2:
        break
    time.sleep(1)
print("Rollout Success")

Rollout Success


In [85]:
!kubectl delete -f resources/fixed_v1_rep2.yaml -n seldon

seldondeployment.machinelearning.seldon.io "fixed" deleted


## Separate Service Orchestrator

We can test that the rolling update works when we use the annotation that allows us to have the service orchestrator on a separate pod, namely `seldon.io/engine-separate-pod: "true"`, as per the config file below. Though in this case both the service orchestrator and model pod will be recreated.

In [86]:
%%writefile resources/fixed_v1_sep.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  annotations:
    seldon.io/engine-separate-pod: "true"
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 1

Overwriting resources/fixed_v1_sep.yaml


In [87]:
!kubectl apply -f resources/fixed_v1_sep.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed created


We can wait until the pod is available before starting the rolling update.

In [88]:
!kubectl wait sdep/fixed --for=condition=ready --timeout=120s -n seldon

seldondeployment.machinelearning.seldon.io/fixed condition met


In [89]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

{"data":{"names":[],"ndarray":[1,2,3,4]},"meta":{"requestPath":{"classifier":"seldonio/fixed-model:0.1"}}}


Now we can make a rolling update by changing the version of the docker image we will be updating it for.

In [90]:
%%writefile resources/fixed_v2_sep.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  annotations:
    seldon.io/engine-separate-pod: "true"
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 1

Overwriting resources/fixed_v2_sep.yaml


In [91]:
!kubectl apply -f resources/fixed_v2_sep.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed configured


And we can send requests to confirm that the rolling update is performed without interruptions

In [92]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 1:
        break
    time.sleep(1)
print("Rollout Success")

Rollout Success


In [93]:
!kubectl delete -f resources/fixed_v1_sep.yaml -n seldon

seldondeployment.machinelearning.seldon.io "fixed" deleted


## Two PodSpecs

We can test that the rolling update works when we have multiple podSpecs in our deployment and only does a rolling update the first pod (which also contains the service orchestrator)

In [94]:
%%writefile resources/fixed_v1_2podspecs.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier1
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier2
    graph:
      name: classifier1
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: default
    replicas: 1

Overwriting resources/fixed_v1_2podspecs.yaml


In [95]:
!kubectl apply -f resources/fixed_v1_2podspecs.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed created


In [96]:
!kubectl wait sdep/fixed --for=condition=ready --timeout=120s -n seldon

seldondeployment.machinelearning.seldon.io/fixed condition met


We can wait until the pod is available before starting the rolling update.

In [97]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

{"data":{"names":[],"ndarray":[1,2,3,4]},"meta":{"requestPath":{"classifier1":"seldonio/fixed-model:0.1","classifier2":"seldonio/fixed-model:0.1"}}}


Now we can make a rolling update by changing the version of the docker image we will be updating it for.

In [98]:
%%writefile resources/fixed_v2_2podspecs.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier1
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier2
    graph:
      name: classifier1
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: default
    replicas: 1

Overwriting resources/fixed_v2_2podspecs.yaml


In [99]:
!kubectl apply -f resources/fixed_v2_2podspecs.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed configured


And we can send requests to confirm that the rolling update is performed without interruptions

In [100]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 1:
        break
    time.sleep(1)
print("Rollout Success")

Failed to parse json ['upstream connect error or disconnect/reset before headers. reset reason: remote connection failure, transport failure reason: delayed connect error: Connection refused']
Failed to parse json ['upstream connect error or disconnect/reset before headers. reset reason: remote connection failure, transport failure reason: delayed connect error: Connection refused']
Failed to parse json ['upstream connect error or disconnect/reset before headers. reset reason: remote connection failure, transport failure reason: delayed connect error: Connection refused']
Failed to parse json ['upstream connect error or disconnect/reset before headers. reset reason: remote connection failure, transport failure reason: delayed connect error: Connection refused']
Failed to parse json ['upstream connect error or disconnect/reset before headers. reset reason: remote connection failure, transport failure reason: delayed connect error: Connection refused']
Rollout Success
Rollout Success


In [101]:
!kubectl delete -f resources/fixed_v1_2podspecs.yaml -n seldon

seldondeployment.machinelearning.seldon.io "fixed" deleted


## Two Models

We can test that the rolling update works when we have two predictors / models in our deployment.

In [102]:
%%writefile resources/fixed_v1_2models.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
        - image: seldonio/fixed-model:0.1
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: default
    replicas: 3

Overwriting resources/fixed_v1_2models.yaml


In [103]:
!kubectl apply -f resources/fixed_v1_2models.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed created


In [104]:
!kubectl wait sdep/fixed --for=condition=ready --timeout=120s -n seldon

seldondeployment.machinelearning.seldon.io/fixed condition met


In [105]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

{"data":{"names":[],"ndarray":[1,2,3,4]},"meta":{"requestPath":{"classifier":"seldonio/fixed-model:0.1","classifier2":"seldonio/fixed-model:0.1"}}}


Now we can make a rolling update by changing the version of the docker image we will be updating it for.

In [106]:
%%writefile resources/fixed_v2_2models.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier
        - image: seldonio/fixed-model:0.2
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: default
    replicas: 3

Overwriting resources/fixed_v2_2models.yaml


In [107]:
!kubectl apply -f resources/fixed_v2_2models.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed configured


And we can send requests to confirm that the rolling update is performed without interruptions

In [108]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 3:
        break
    time.sleep(1)
print("Rollout Success")

Rollout Success


In [109]:
!kubectl delete -f resources/fixed_v2_2models.yaml -n seldon

seldondeployment.machinelearning.seldon.io "fixed" deleted


## Two Predictors

We can test that the rolling update works when we have two predictors in our deployment.

In [110]:
%%writefile resources/fixed_v1_2predictors.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
        - image: seldonio/fixed-model:0.1
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: a
    replicas: 3
    traffic: 50
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
        - image: seldonio/fixed-model:0.1
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: b
    replicas: 1
    traffic: 50

Overwriting resources/fixed_v1_2predictors.yaml


In [111]:
!kubectl apply -f resources/fixed_v1_2predictors.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed created


We can wait until the pod is available before starting the rolling update.

In [112]:
!kubectl wait sdep/fixed --for=condition=ready --timeout=120s -n seldon

seldondeployment.machinelearning.seldon.io/fixed condition met


In [113]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

{"data":{"names":[],"ndarray":[1,2,3,4]},"meta":{"requestPath":{"classifier":"seldonio/fixed-model:0.1","classifier2":"seldonio/fixed-model:0.1"}}}


Now we can make a rolling update by changing the version of the docker image we will be updating it for.

In [114]:
%%writefile resources/fixed_v2_2predictors.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier
        - image: seldonio/fixed-model:0.2
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: a
    replicas: 3
    traffic: 50
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
        - image: seldonio/fixed-model:0.1
          name: classifier2
    graph:
      name: classifier
      type: MODEL
      children:
      - name: classifier2
        type: MODEL
    name: b
    replicas: 1
    traffic: 50

Overwriting resources/fixed_v2_2predictors.yaml


In [115]:
!kubectl apply -f resources/fixed_v2_2predictors.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed configured


And we can send requests to confirm that the rolling update is performed without interruptions

In [116]:
time.sleep(5)  # To allow operator to start the update
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numReplicas = int(resources["items"][0]["status"]["replicas"])
    if numReplicas == 3:
        break
    time.sleep(1)
print("Rollout Success")

Failed to parse json ['upstream connect error or disconnect/reset before headers. reset reason: remote connection failure, transport failure reason: delayed connect error: Connection refused']
Rollout Success
Rollout Success


In [117]:
!kubectl delete -f resources/fixed_v2_2predictors.yaml -n seldon

seldondeployment.machinelearning.seldon.io "fixed" deleted


## Model name changes

This will not do a rolling update but create a new deployment.

In [118]:
%%writefile resources/fixed_v1.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.1
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 3

Overwriting resources/fixed_v1.yaml


In [119]:
!kubectl apply -f resources/fixed_v1.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed created


We can wait until the pod is available.

In [120]:
!kubectl wait sdep/fixed --for=condition=ready --timeout=120s -n seldon

seldondeployment.machinelearning.seldon.io/fixed condition met


In [121]:
!curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions \
   -H "Content-Type: application/json"

{"data":{"names":[],"ndarray":[1,2,3,4]},"meta":{"requestPath":{"classifier":"seldonio/fixed-model:0.1"}}}


Now when we apply the update, we should see the change taking place, but there should not be an actual full rolling update triggered.

In [122]:
%%writefile resources/fixed_v2_new_name.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: fixed
spec:
  name: fixed
  protocol: seldon
  transport: rest
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/fixed-model:0.2
          name: classifier2
    graph:
      name: classifier2
      type: MODEL
    name: default
    replicas: 3

Writing resources/fixed_v2_new_name.yaml


In [123]:
!kubectl apply -f resources/fixed_v2_new_name.yaml -n seldon

seldondeployment.machinelearning.seldon.io/fixed configured


In [124]:
time.sleep(5)
for i in range(120):
    responseRaw = !curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' -X POST http://localhost:8003/seldon/seldon/fixed/api/v1.0/predictions -H "Content-Type: application/json"
    try:
        response = json.loads(responseRaw[0])
    except:
        print("Failed to parse json", responseRaw)
        continue
    assert response["data"]["ndarray"][0] == 1 or response["data"]["ndarray"][0] == 5
    jsonRaw = !kubectl get deploy -l seldon-deployment-id=fixed -o json
    data = "".join(jsonRaw)
    resources = json.loads(data)
    numItems = len(resources["items"])
    if numItems == 1:
        break
    time.sleep(1)
print("Rollout Success")

Rollout Success


In [125]:
!kubectl delete -f resources/fixed_v2_new_name.yaml -n seldon

seldondeployment.machinelearning.seldon.io "fixed" deleted
